# Объяснение метода

In [91]:
import numpy as np
import sympy as sym
from sympy import exp

x0 = np.array([0, 0], dtype=float)

# переменные
x, y = sym.symbols("x y")

# функция 7 варианта
# f = 10 - (x - 3) * exp(-x + 3) - (y - 2) * exp(-x + 2)
# функция с методички
f = x**2 + x*y + 2*y**2 + x - y

f

x**2 + x*y + x + 2*y**2 - y

In [92]:
lmbd = sym.symbols('lmbd')

variables = (x,y)
variables

(x, y)

In [93]:
# замена каждой переменной на [переменная + lmbd]
f_replaced = [f.subs(variable, variable + lmbd) 
              for variable in variables]

f_replaced

[lmbd + x + 2*y**2 + y*(lmbd + x) - y + (lmbd + x)**2,
 -lmbd + x**2 + x*(lmbd + y) + x - y + 2*(lmbd + y)**2]

In [94]:
# градиент по lmbd
f_replaced_grad = [sym.diff(f, lmbd) for f in f_replaced]

f_replaced_grad

[2*lmbd + 2*x + y + 1, 4*lmbd + x + 4*y - 1]

In [95]:
# выражение n штук lmbd через n переменных функции f
lmbds = [sym.solve(f, lmbd)[0] for f in f_replaced_grad]

lmbds

[-x - y/2 - 1/2, -x/4 - y + 1/4]

In [96]:
# превращение в лямбда-функцию
lmbds = [sym.lambdify(variables, lmbd) for lmbd in lmbds]

lmbds

[<function _lambdifygenerated(x, y)>, <function _lambdifygenerated(x, y)>]

In [97]:
# 1 итерация
x1 = x0.copy()

lmbd1 = lmbds[0](*x1)

x1[0] += lmbd1

x1, x0

(array([-0.5,  0. ]), array([0., 0.]))

In [98]:
lmbd2 = lmbds[1](*x1)

x1[1] += lmbd2

x1, x0

(array([-0.5  ,  0.375]), array([0., 0.]))

In [99]:
# проверка на выход из алгоритма
norm = np.linalg.norm

eps = 0.1

norm(x1 - x0), norm(x1 - x0) < eps

(0.625, False)

# Метод Гаусса-Зейделя

In [100]:
import sympy as sym
import numpy as np
norm = np.linalg.norm

def gauss_seidel_method(
    variables: tuple[sym.Symbol],
    f: sym.Add,
    x0: np.ndarray,
    eps: float
) -> float:
    lmbd = sym.symbols('lmbd')

    # замена каждой переменной на [переменная + lmbd]
    f_replaced = [f.subs(variable, variable + lmbd) 
                  for variable in variables]
    
    print(f_replaced)
    
    # градиент по lmbd
    f_replaced_grad = [sym.diff(f, lmbd) 
                       for f in f_replaced]

    # выражение n штук lmbd через n переменных функции f
    lmbds = [sym.solve(f, lmbd)[0] 
             for f in f_replaced_grad]

    # превращение выражений в лямбда-функции
    lmbds = [sym.lambdify(variables, lmbd) 
             for lmbd in lmbds]
    
    x1 = x0.copy()

    # типо while true
    for _ in range(10**4):
        for i in range(len(lmbds)):
            x1[i] += lmbds[i](*x1)
        print(f'итерация {_}: {x1}')
        
        if norm(x1 - x0) < eps:
            return x1
        x0 = x1.copy()

In [101]:
x, y = sym.symbols('x y')

# функция 7 варианта
# f = 10 - (x - 3) * exp(-x + 3) - (y - 2) * exp(-x + 2)

# функция с методочки
f = x**2 + x*y + 2*y**2 + x - y

x0 = np.array([0, 0], dtype=float)

eps = 0.1

# answer -5/7 и 3/7
gauss_seidel_method((x, y), f, x0, eps)

[lmbd + x + 2*y**2 + y*(lmbd + x) - y + (lmbd + x)**2, -lmbd + x**2 + x*(lmbd + y) + x - y + 2*(lmbd + y)**2]
итерация 0: [-0.5    0.375]
итерация 1: [-0.6875    0.421875]
итерация 2: [-0.7109375   0.42773438]


array([-0.7109375 ,  0.42773438])